In [1]:
import json
import numpy as np
from shutil import copyfile
import os

In [ ]:
with open('../data/bdd100k/labels/bdd100k_labels_images_train.json',) as f:
    data = json.load(f)
with open('../data/bdd100k/labels/bdd100k_labels_images_val.json',) as f:
    val_data = json.load(f)

In [ ]:
data += val_data

In [ ]:
len(data)

In [2]:
def split_data(path, filename, split_perc=[0.8, 0.1, 0.1]):
    assert np.sum(split_perc) == 1.
    file = open(path + filename + '.txt', 'r')
    names = [line.strip('\n') for line in file]
    names = np.random.permutation(names)
    
    split_len = np.array(split_perc) * len(names)
    split_idx = split_len[:-1].astype(int)
    split_idx[1] += split_idx[0]
    sub_arrays = np.split(names, split_idx)
    
    ftrain = open(path + filename.strip('_all') + '_train.txt', 'w+')
    fval = open(path + filename.strip('_all') + '_val.txt', 'w+')
    ftest = open(path + filename.strip('_all') + '_test.txt', 'w+')
    
    for array, file in zip(sub_arrays, [ftrain, fval, ftest]):
        for name in array:
            file.write(name + '\n')
        
    ftrain.close()
    fval.close()
    ftest.close()
    
split_data('../PyTorch-YOLOv3/data/custom/', 'daytime_all')

In [ ]:
def keep_conditions():
    ftxt = open("../PyTorch-YOLOv3/data/custom/daytime.txt", "w+")
    count = 0

    for image in data:
        iname = image['name']

        if image['attributes']['timeofday'] == 'daytime':
            ftxt.write('data/custom/images/' + iname + '\n')
            count += 1
    print(count)

In [ ]:
def copy_all_images():
    ftxt = open("../PyTorch-YOLOv3/data/custom/all.txt", "w+")

    for image in data:
        iname = image['name']
        fname = iname[:-4]
        f = open("../PyTorch-YOLOv3/data/custom/labels/" + fname + ".txt","w+")

        labels = image['labels']
        for label in labels:
            if 'box2d' not in label:
                continue
            xy = label['box2d']
            if xy['x1'] >= xy['x2'] or xy['y1'] >= xy['y2']:
                continue
            X = xy['x1'] / 1280
            Y = xy['y1'] / 720
            MX = ((xy['x1'] + xy['x2']) / 2) / 1280
            MY = ((xy['y1'] + xy['y2']) / 2) / 720
            W = xy['x2'] / 1280
            H = xy['y2'] / 720
            if X > W or Y > H:
                continue

            lbl = -1
            if(label['category'] == "bike"):
                lbl = 0
            if(label['category'] == "bus"):
                lbl = 1
            if(label['category'] == "car"):
                lbl = 2
            if(label['category'] == "motor"):
                lbl = 3
            if(label['category'] == "person"):
                lbl = 4
            if(label['category'] == "rider"):
                lbl = 5
            if(label['category'] == "traffic light"):
                lbl = 6
            if(label['category'] == "traffic sign"):
                lbl = 7
            if(label['category'] == "train"):
                lbl = 8
            if(label['category'] == "truck"):
                lbl = 9

            f.write(repr(lbl) + " " + repr(MX) + " " + repr(MY) + " " + repr(W-X) + " " + repr(H-Y) + '\n')
            ftxt.write('data/custom/images/' + iname + '\n')
    #         copyfile('../data/bdd100k/images/100k/val/' + iname, '../PyTorch-YOLOv3/data/custom/images/' + iname) 